In [9]:
import numpy as np
import os
import h5py
import skvideo.io
import logging

from pathlib import Path
from typing import List

In [10]:
# set up logging
logging.basicConfig(level=logging.INFO)

In [11]:
def find_all_series(data_folders: List[str]) -> List[str]:
    """Find all .h5 series from a list of folders.
    
    Args:
        data_folders: List of paths to folders containing .h5 series.
    
    Returns:
        A list of filenames to .h5 series.
    """
    h5_series = []
    for data_folder in data_folders:
        # h5_series.extend([Path(p).as_posix() for p in glob(f"{data_folder}/*.h5")])
        h5_series.extend([p.as_posix() for p in Path(data_folder).rglob("*.h5")])
    return h5_series


def render_plant_from_h5(h5_path, save_path, decimate=4, slowdown_factor=1, x0=512, x1=1536, overwrite=False):
    """Create an MP4 video from a dataset within an H5 file.
    
    Args:
        h5_path: Path to the H5 file containing the 'vol' dataset with images.
        save_path: Path where the output MP4 video will be saved.
        decimate: Factor by which the images will be decimated (reduced in resolution).
        slowdown_factor: Factor by which the video playback will be slowed down.
        x0, x1: Horizontal bounds for cropping the image.
        overwrite: If False, existing videos will not be overwritten.
    
    Returns:
        None. The video is saved to the specified path if it does not exist or overwrite is True.
    """
    save_path = Path(save_path)
    if save_path.exists() and not overwrite:
        logging.info(f"Video {save_path} already exists. Skipping due to overwrite=False.")
        return
    
    logging.info(f"Opening H5 file: {h5_path}")
    with h5py.File(h5_path, 'r') as f:
        logging.info("Reading 'vol' dataset from H5 file...")
        data = f['vol'][:]
        logging.info(f"Dataset shape: {data.shape}")
    
    out_video = []
    logging.info("Processing images...")
    for i, img in enumerate(data):
        # Crop and decimate
        # img = img[:, x0:x1]
        img = img[::decimate, ::decimate]
        if i % 10 == 0:  # logging.info progress every 10 frames
            logging.info(f"Processed {i+1}/{len(data)} frames...")
        out_video.append(img)
    
    out_video = np.stack(out_video, axis=0)
    logging.info(f"Total frames before slowdown: {len(out_video)}")
    
    if slowdown_factor > 1:
        out_video = np.repeat(out_video, slowdown_factor, axis=0)
        logging.info(f"Applied slowdown factor. Total frames after slowdown: {len(out_video)}")
    
    # Ensure the video is in the correct format for skvideo to write
    out_video = out_video.astype(np.uint8)
    
    logging.info(f"Writing video to {save_path}...")
    skvideo.io.vwrite(str(save_path), out_video)
    logging.info("Video creation completed.")

In [12]:
# Path to find h5s and save mp4s (1 per h5)
base_dir = "H:/users/eberrigan/20250225_Elohim_Bello_Exp02_timelapse_images/Arabidopsis_time_lapse_videos_one_frame_per_day_20250227/h5s_preds_by_frame"

In [13]:
# If overwrite=True mp4s will be overwritten
overwrite = True
decimation = 2 # Factor used to coarsen the video 
# x0, x1 = (550, 1630) # cropping
# x1 = ((x1 - x0) // (4 * decimation)) * (4 * decimation) + x0 # adjusted cropping
slowdown_factor = 5  # must be integer >= 1

In [14]:
# Make sure the base directory is a Path object
base_dir = Path(base_dir)

# Check current working directory
logging.info(f"Current working directory: {os.getcwd()}")
logging.info(f"Looking in: {base_dir.absolute()}")  # logging.info the absolute path for clarity

# Ensure the directory exists
if not base_dir.exists():
    logging.info(f"The directory {base_dir.absolute()} does not exist.")
else:
    # Attempt to find and process .h5 files
    # Find all .h5 files within the base directory and its subdirectories
    for h5_file in base_dir.glob('**/*.h5'):
        mp4_file = h5_file.with_suffix('.mp4')
        logging.info(f"Processing {h5_file}...")
        render_plant_from_h5(h5_file, mp4_file, decimate=decimation, slowdown_factor=slowdown_factor, x0=512, x1=1536, overwrite=overwrite)
        logging.info(f"Made {mp4_file}...")

INFO:root:Current working directory: c:\repos\cylinder-time-lapse
INFO:root:Looking in: H:\users\eberrigan\20250225_Elohim_Bello_Exp02_timelapse_images\Arabidopsis_time_lapse_videos_one_frame_per_day_20250227\h5s_preds_by_frame
INFO:root:Processing H:\users\eberrigan\20250225_Elohim_Bello_Exp02_timelapse_images\Arabidopsis_time_lapse_videos_one_frame_per_day_20250227\h5s_preds_by_frame\g1_r1.h5...
INFO:root:Opening H5 file: H:\users\eberrigan\20250225_Elohim_Bello_Exp02_timelapse_images\Arabidopsis_time_lapse_videos_one_frame_per_day_20250227\h5s_preds_by_frame\g1_r1.h5
INFO:root:Reading 'vol' dataset from H5 file...
INFO:root:Dataset shape: (18, 1080, 2048, 1)
INFO:root:Processing images...
INFO:root:Processed 1/18 frames...
INFO:root:Processed 11/18 frames...
INFO:root:Total frames before slowdown: 18
INFO:root:Applied slowdown factor. Total frames after slowdown: 90
INFO:root:Writing video to H:\users\eberrigan\20250225_Elohim_Bello_Exp02_timelapse_images\Arabidopsis_time_lapse_vide